# Trace Upstream Script 
## Version 2: Directly create upstream trib feature classes 
Kernel: Python 2

Developed by: Curtis Fang

In [1]:
import arcpy
import numpy as np
import pandas as pd

### Set GDB directory

In [33]:
# arcpy.env.workspace='P:\GIS\Projects\LARiver_Watershed_Projects\General\Shapefiles\Subwatersheds\SubWSTributary.gdb'
arcpy.env.workspace='Input\LosPen\CC_With Downstream_IDs\output.gdb'
arcpy.env.overwriteOutput = True
# input_data='lac_SubWSCatchment_LARWSClip'
input_data='DownIDCat'

### Import input feature class as a pandas dataframe 

In [34]:
arcpy.conversion.TableToExcel(Input_Table=input_data,
                              Output_Excel_File='Output/input.xls'
                             )
df = pd.read_excel('output/input.xls')

In [35]:
orig_cols=df.columns
orig_cols

Index([u'OBJECTID_1', u'Watershed', u'Final_SWID', u'Shape_Leng', u'Sub_ID',
       u'ObjectID', u'Sub_ID_1', u'Dwn_Sub_ID', u'Shape_Length', u'Shape_Area',
       u'Sub_ID_s', u'Dwn_Sub_ID_s'],
      dtype='object')

### Set target downstream catchment and catchment ID columns 

In [36]:
ID_list = [
's4012352145'
]

In [37]:
id_col='Sub_ID_s'
ds_col='Dwn_Sub_ID_s'
for target_ID in ID_list:
    # Define output variables
    temp_input="_temp_input_"+target_ID
    temp_input_xls='Output/'+temp_input+'.xls'
    table_gdb='a'+target_ID+'_upstream'
    table_csv=filename = 'Output/'+table_gdb+'.csv'
    fc_output='_upstream_'+target_ID

    # Trace upstream
    _neighbors = df[df[ds_col] == target_ID]
    upstream = _neighbors
    while not _neighbors.empty:
        _neighbors = df[df[ds_col].isin(_neighbors[id_col])]
#         print(_neighbors.shape)
        upstream = upstream.append(_neighbors, ignore_index=True)
#         print(upstream.shape)
    #Comment out the line below if one does not want to include the catchment itself
    upstream=upstream.append(df[df[id_col]==target_ID],ignore_index=True)
    
    exc_cols=orig_cols.difference([id_col])
    upstream = upstream.drop(exc_cols,1)
    
    upstream.to_csv(table_csv,index=False)
    arcpy.conversion.TableToTable(table_csv,
                                 arcpy.env.workspace,
                                table_gdb)
    
    #Create a temp duplicate shapefile in prep for join field - avoid modifying the original input
    arcpy.conversion.FeatureClassToFeatureClass(input_data, 
                                            arcpy.env.workspace, 
                                            temp_input)
    
    arcpy.management.JoinField(in_data=temp_input,
                          in_field=id_col,
                          join_table= table_gdb,
                          join_field=id_col)
    
    expression = id_col+'_1 IS NOT NULL' #Furture improvement: Avoid hard coding the epxression
    arcpy.conversion.FeatureClassToFeatureClass(temp_input, 
                                            arcpy.env.workspace, 
                                            fc_output,
                                           expression)
    arcpy.management.Delete(temp_input)
    arcpy.management.DeleteField(fc_output,id_col+'_1')
    


### Define output variables

In [ ]:
temp_input="_temp_input_"+target_ID
temp_input_xls='Output/'+temp_input+'.xls'
table_gdb='a'+target_ID+'_upstream'
table_csv=filename = 'Output/'+table_gdb+'.csv'
fc_output='upstream_'+target_ID

### Trace upstream

In [ ]:
_neighbors = df[df[ds_col] == target_ID]
upstream = _neighbors
while not _neighbors.empty:
    _neighbors = df[df[ds_col].isin(_neighbors[id_col])]
    upstream = upstream.append(_neighbors, ignore_index=True)
#Comment out the line below if one does not want to include the catchment itself
upstream=upstream.append(df[df[id_col]==target_ID],ignore_index=True)

In [ ]:
exc_cols=orig_cols.difference([id_col])
upstream = upstream.drop(exc_cols,1)
upstream

In [ ]:
upstream.to_csv(table_csv,index=False)
arcpy.conversion.TableToTable(table_csv,
                             arcpy.env.workspace,
                            table_gdb)

In [ ]:
#Create a temp duplicate shapefile in prep for join field - avoid modifying the original input
arcpy.conversion.FeatureClassToFeatureClass(input_data, 
                                            arcpy.env.workspace, 
                                            temp_input)

In [ ]:
arcpy.management.JoinField(in_data=temp_input,
                          in_field=id_col,
                          join_table= table_gdb,
                          join_field=id_col)

In [ ]:
# Check join result
arcpy.conversion.TableToExcel(Input_Table=temp_input,Output_Excel_File=temp_input_xls)
df3 = pd.read_excel(temp_input_xls)
df3

In [ ]:
expression = id_col+'_2018 IS NOT NULL' #Furture improvement: Avoid hard coding the epxression
arcpy.conversion.FeatureClassToFeatureClass(temp_input, 
                                            arcpy.env.workspace, 
                                            fc_output,
                                           expression)

In [ ]:
#if run into already exist error, run this block
#arcpy.management.Delete(fc_output)
# arcpy.env.overwriteOutput = True

### Clean up temp files 

In [ ]:
arcpy.management.Delete(temp_input)
arcpy.management.DeleteField(fc_output,id_col+'_1')